<a href="https://colab.research.google.com/github/Alaa-alsulaiman/Week_9/blob/main/W9A3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
import hashlib
import json
import time
tx_1 = {
   "addr_from": "3J98t1WpEZ73CNmQviecrnyiWrnqRhWNLy",
   "addr_to": "1BvBMSEYstWetqTFn5Au4m4GFg7xJaNVN2",   
   "amount": 99.00
}
tx_2 = {
   "addr_from": "1BvBMSEYstWetqTFn5Au4m4GFg7xJaNVN2",
   "addr_to": "3J98t1WpEZ73CNmQviecrnyiWrnqRhWNLy",   
   "amount": 97.00
}

In [67]:
class Block:
    def __init__(self,index:int =0,transactions:list = [],timestamp:int = 0,cur_hash:str = "",previous_hash:str ="",nonce:int =0):
        self.index:int = index
        self.transactions:list = transactions
        self.timestamp:float = timestamp
        self.hash:str = cur_hash
        self.previous_hash:str = previous_hash
        self.nonce:int = nonce
    
    def compute_hash(self) -> str:
        block_string = json.dumps(self.__dict__, sort_keys=True)
        return hashlib.sha256(block_string.encode()).hexdigest()


In [78]:
class Blockchain: 
    def __init__(self):
        self.unconfirmed_transactions:list[dict] = [] #mempool
        self.chain:list[Block] = []
        self.difficulty:int = 2
        self.create_genesis_block()

    def get_last_block(self)-> Block:
          return self.chain[-1]
 
    def create_genesis_block(self):
        genesis_block = Block(index=0, transactions=[], timestamp=time.time(), previous_hash="0")
        genesis_block.hash = genesis_block.compute_hash()
        self.chain.append(genesis_block)

    def proof_of_work(self, block:Block) -> str:
          computed_hash = block.compute_hash()

          while not computed_hash.startswith('0' * self.difficulty):
               block.nonce += 1
               computed_hash = block.compute_hash()
          return computed_hash 
   
    def is_valid_proof(self, block:Block, block_hash:str)-> bool:
      return block_hash.startswith('0' * self.difficulty) and block_hash ==  block.compute_hash()

    def add_block(self, block:Block, proof:str) -> bool:
        last_block = self.get_last_block()
        last_block.hash

        if last_block.hash != block.previous_hash:
            return False
        if not self.is_valid_proof(block, proof):
            return False

        block.hash = proof
        self.chain.append(block)
        return True 


    def add_new_transaction(self, transaction):
            self.unconfirmed_transactions.append(transaction)


    def mine(self) -> int:
          if len(self.unconfirmed_transactions)<1:
            return -1      

          last_block = self.get_last_block()
          self.unconfirmed_transactions.append(tx_1)

          new_block = Block(index=last_block.index+1,transactions=self.unconfirmed_transactions,timestamp=int(time.time()),previous_hash=last_block.hash)
        
          proof = self.proof_of_work(new_block)

          self.add_block(new_block, proof)

          self.unconfirmed_transactions = []

          return new_block.index
          

In [79]:
bc = Blockchain()
[bc.add_new_transaction(tx) for tx in [tx_1,tx_2]]

[None, None]

In [80]:
bc.unconfirmed_transactions

[{'addr_from': '3J98t1WpEZ73CNmQviecrnyiWrnqRhWNLy',
  'addr_to': '1BvBMSEYstWetqTFn5Au4m4GFg7xJaNVN2',
  'amount': 99.0},
 {'addr_from': '1BvBMSEYstWetqTFn5Au4m4GFg7xJaNVN2',
  'addr_to': '3J98t1WpEZ73CNmQviecrnyiWrnqRhWNLy',
  'amount': 97.0}]

In [81]:
bc.get_last_block().__dict__

{'hash': 'ee139fc559aa0a1dd05ac2e02e1cd572ad227bd9443badf752b1452600a6203b',
 'index': 0,
 'nonce': 0,
 'previous_hash': '0',
 'timestamp': 1639329918.5461316,
 'transactions': []}

In [82]:
bc.mine()

1